<center> 
    <h1> Steganography Project </h1> 
    <h3> CS07552 - Cryptographic Algorithms </h3>
    <h3> Benny Liang </h3>
</center>

### Importing libraries

In [1]:
import os
import sys
import re
from typing import Union
from PIL import Image
import numpy as np
import hashlib
import base64
from random import seed, randint
from Crypto import Random
from Crypto.Cipher import AES
from Crypto.Hash import SHA512, SHA384, SHA256, SHA224
from cryptography.hazmat.primitives import padding
import cv2

print("System information: {0}".format(sys.version))

System information: 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]


### Input variables

In [2]:
BLOCK_SIZE = 128
MAX_NUM_BLOCKS_LENGTH = 16
ORIGINAL_IMAGE_PATH = "C:/Users/pawan/Desktop/Crytographic_algorithm/original.png"
MODIFIED_IMAGE_DIR = "C:/Users/pawan/Desktop/Crytographic_algorithm/"
PASSPHRASE = "CXSOToq3oY0r9wm1P0T1"
MESSAGE = "Two households, both alike in dignity, In fair Verona, where we lay our scene, From ancient grudge break to new mutiny, Where civil blood makes civil hands unclean. From forth the fatal loins of these two foes A pair of star-cross'd lovers take their life; Whose misadventured piteous overthrows Do with their death bury their parents' strife. The fearful passage of their death-mark'd love, And the continuance of their parents' rage, Which, but their children's end, nought could remove, Is now the two hours' traffic of our stage; The which if you with patient ears attend, What here shall miss, our toil shall strive to mend.Two households, both alike in dignity, In fair Verona, where we lay our scene, From ancient grudge break to new mutiny, Where civil blood makes civil hands unclean. From forth the fatal loins of these two foes A pair of star-cross'd lovers take their life; Whose misadventured piteous overthrows Do with their death bury their parents' strife. The fearful passage of their death-mark'd love, And the continuance of their parents' rage, Which, but their children's end, nought could remove, Is now the two hours' traffic of our stage; The which if you with patient ears attend, What here shall miss, our toil shall strive to mend.Two households, both alike in dignity, In fair Verona, where we lay our scene, From ancient grudge break to new mutiny, Where civil blood makes civil hands unclean. From forth the fatal loins of these two foes A pair of star-cross'd lovers take their life; Whose misadventured piteous overthrows Do with their death bury their parents' strife. The fearful passage of their death-mark'd love, And the continuance of their parents' rage, Which, but their children's end, nought could remove, Is now the two hours' traffic of our stage; The which if you with patient ears attend, What here shall miss, our toil shall strive to mend.Two households, both alike in dignity, In fair Verona, where we lay our scene, From ancient grudge break to new mutiny, Where civil blood makes civil hands unclean. From forth the fatal loins of these two foes A pair of star-cross'd lovers take their life; Whose misadventured piteous overthrows Do with their death bury their parents' strife. The fearful passage of their death-mark'd love, And the continuance of their parents' rage, Which, but their children's end, nought could remove, Is now the two hours' traffic of our stage; The which if you with patient ears attend, What here shall miss, our toil shall strive to mend.Two households, both alike in dignity, In fair Verona, where we lay our scene, From ancient grudge break to new mutiny, Where civil blood makes civil hands unclean. From forth the fatal loins of these two foes A pair of star-cross'd lovers take their life; Whose misadventured piteous overthrows Do with their death bury their parents' strife. The fearful passage of their death-mark'd love, And the continuance of their parents' rage, Which, but their children's end, nought could remove, Is now the two hours' traffic of our stage; The which if you with patient ears attend, What here shall miss, our toil shall strive to mend."

MESSAGE = MESSAGE.encode('utf-8')
PASSPHRASE = PASSPHRASE.encode('utf-8')
IMAGE_NAME = re.search(r'(\.*?).png', ORIGINAL_IMAGE_PATH).group(1)
MODIFIED_IMAGE_PATH = "{0}{1}modified.png".format(MODIFIED_IMAGE_DIR, IMAGE_NAME)

In [3]:
with open('input.txt', 'rb') as file:
    MESSAGE = file.read()

### Viewing image details

In [4]:
image = Image.open("C:/Users/pawan/Desktop/Crytographic_algorithm/original.png")
assert image.format == "PNG", "Please use a PNG instead of a {0:s}".format(image.format)
assert image.mode == "RGB" or image.mode == "RGBA", "Please use a RGB image instead of a {0:s}".format(image.mode)
print(image)

<PIL.PngImagePlugin.PngImageFile image mode=RGB size=509x340 at 0x1F4825DF8C8>


### Generating the AES key and permutation key

In [5]:
def get_shaHash(data:Union[str, bytes], algorithm:int) -> hex:
    """
    Returns a hex representation of the hashed data using SHA224, SHA256, SHA384, or SHA512
    Arguments:
    data: \t the inputted data to be hashed. Expected types: str, and bytes. 
    algorithm: \t int representation of which SHA algorithm to use. Will defaults to SHA384.
    """
    shaHash = None
    try:
        if algorithm == 512:
            shaHash = SHA512.new()
        elif algorithm == 384:
            shaHash = SHA384.new()
        elif algorithm == 256:
            shaHash = SHA256.new()
        elif algorithm == 224:
            shaHash = SHA224.new()
    except:
        pass
        
    if shaHash == None:
        shaHash = SHA384.new()
    
    shaHash = SHA384.new()
    if isinstance(data, str):
        try:
            bytes(data).decode('utf-8')
        except (TypeError, UnicodeDecodeError) as exception:
            data = data.encode('utf-8')
    shaHash.update(data)
    return shaHash.hexdigest()


def bytes_toInt(data:bytes) -> int:
    if isinstance(data, bytes):
        return int.from_bytes(data, 'big')
    raise TypeError("key is of type {0}, but should be bytes".format(type(key)))

def int_toBytes(data:int):
    if isinstance(data, int):
        return data.to_bytes((data.bit_length() + 7) // 8, 'big')
    elif isinstance(data, bytes):
        return data
    raise TypeError("data is of type {0}, but should be int.".format(type(key)))

# Finds the SHA hash of the passphrase
sha_hash = get_shaHash(PASSPHRASE, 384)

# SHA hash is broken up into two parts, one is used for AES encryption while the other is used in permutation
# Both of these keys are stored as bytes
aes_key = int_toBytes(int(sha_hash[0:64], 16))
permutation_seed = int_toBytes(int(sha_hash[64:], 16))

assert int(sha_hash[0:64], 16) == bytes_toInt(aes_key), "bytes_toInt(key) does not work correctly."

In [6]:
c = sha_hash
print(c)
d = int(c, 16)
print(d)
e = d % (2**32 - 1)
print(e)
np.random.seed(e)

d80816bd0eb4eaa467aff42743551d2c79cae730ddb158a53b28bf110173f46ccd5f393e3b31479ee3e257ae2b187eea
33250305945766165614567972043915182779977822373536008717443449554706873132236606284842258673267338903514434791571178
1030118783


### Encrypting the message with AES

In [21]:
def aes_encrypt(data:Union[str, bytes], key:bytes) -> str:
    """
    Returns a string for an encrypted mesasge using the AES algorithm.
    The message is padded with PKCS7 to work with AES.
    Arguments:
    data: \t The data to be encrypted. Expected types of string or bytes.
    key: \t The key to be used with the AES algorithm. Expected type of bytes.
    """
    cipher = AES.new(key, AES.MODE_ECB)
    padder = padding.PKCS7(BLOCK_SIZE).padder()
    if isinstance(data, str):
        data = bytes(data)
    padded_data = padder.update(data)
    padded_data += padder.finalize()
    return cipher.encrypt(padded_data)



In [ ]:
def aes_decrypt(encrypted_data:bytes, key:bytes) -> str:
    """
    Returns a string for the decrypted message using the AES algorithm.
    The padding on the message with PKCS7 is removed.
    Arguments:
    encrypted_data: \t The encrypted data using AES. Expected types of bytes.
    key: \t The key to be used with the AES algorithm. Expected type of bytes.
    """
    cipher = AES.new(key, AES.MODE_ECB)
    padded_data = cipher.decrypt(encrypted_data)
    unpadder = padding.PKCS7(BLOCK_SIZE).unpadder()
    data = unpadder.update(padded_data)
    data += unpadder.finalize()
    return data

In [8]:
assert aes_decrypt(aes_encrypt(MESSAGE, aes_key), aes_key) == bytes(MESSAGE), "Decrypt(encrypted_data) does not match the original data."

encrypted_data = aes_encrypt(MESSAGE, aes_key)
num_data_blocks = int(len(encrypted_data) / (BLOCK_SIZE / 8))
assert num_data_blocks <= 2**MAX_NUM_BLOCKS_LENGTH, "Number of data blocks exceeds capacity."
print(num_data_blocks)


197


### Hiding Encrypted Data in Image

In [9]:
# Reading the image with OpenCV
# Note OpenCV uses the format of pixel = image[y][x]
img = cv2.imread(ORIGINAL_IMAGE_PATH)

num_blocksX = int(image.size[0] / 8)
num_blocksY = int(image.size[1] / 8)
total_blocks = num_blocksX * num_blocksY

assert num_data_blocks + 2 < total_blocks, "Error, data is too big for the inputted image. Number of blocks needed = {0}. Number of blocks available = {1}. Please try again with less data, or a bigger image.".format(num_data_blocks + 2, total_blocks)

# Tracking blocks that have been visited
# -1 = header block
# 0  = not visited
# 1  = visited
blocks_used = np.zeros((num_blocksY, num_blocksX))  #assigning Zeros to all the blocks first

In [10]:
# Establish header blocks

num_iterations = 1

# Using the permutation_seed select the first header block
permutation_value = int(get_shaHash(permutation_seed, 512), 16)
block_index = permutation_value % total_blocks
blockX = block_index % num_blocksX
blockY = int(block_index / num_blocksX)
blocks_used[blockY, blockX] = -1

# Using data from the first block, update the permutation_seed
pixelX_start = blockX * 8
pixelX_curr = pixelX_start

pixelY_start = blockY * 8
pixelY_curr = pixelY_start

print(permutation_value, permutation_seed)

update_seed_data = str(num_iterations)
for i in range(8):
    pixel = img[pixelY_curr + i][pixelX_curr + i]
    update_seed_data = update_seed_data + str(pixel[0] % 100) + str(pixel[1] % 100) + str(pixel[2] % 100)    
permutation_seed = int_toBytes(int(update_seed_data) ^ bytes_toInt(permutation_seed))

29035879622574208841602797001976546678430449296620076083035834650165817742032306179209201031564640884269426194869063 b'\xcd_9>;1G\x9e\xe3\xe2W\xae+\x18~\xea'


In [11]:
# Using the updated permutation seed, find the second header block
found_block = False
while not found_block:
    num_iterations += 1
    permutation_value = int(get_shaHash(permutation_seed, 512), 16)
    block_index = permutation_value % total_blocks
    blockX = block_index % num_blocksX
    blockY = int(block_index / num_blocksX)
    if blocks_used[blockY][blockX] == 0:
        found_block = True
        blocks_used[blockY][blockX] = -1
    else:
        permutation_seed = int_toBytes(bytes_toInt(permutation_seed) + num_iterations)
        
# Hides the number of data blocks there are the pixels 17-31
num_data_blocks = format(num_data_blocks, "#0{0}b".format(MAX_NUM_BLOCKS_LENGTH + 2))
rgb_index = permutation_value % 3

pixelX_start = blockX * 8
pixelX_curr = pixelX_start

pixelY_start = blockY * 8
pixelY_curr = pixelY_start

update_seed_data = str(num_iterations)
for i in range(8):
    pixel = img[pixelY_curr][pixelX_curr + 1]
    update_seed_data = update_seed_data + str(pixel[0] % 100) + str(pixel[1] % 100) + str(pixel[2] % 100)

pixelY_curr += 2
for char in num_data_blocks[2:]:
    pixel = img[pixelY_curr][pixelX_curr]
    pixel[rgb_index] = pixel[rgb_index] & ~1 | int(char)
    img[pixelY_curr][pixelX_curr] = pixel
    rgb_index = (rgb_index + 1) % 3
    pixelX_curr += 1
    if pixelX_curr % 8 == 0:
        pixelX_curr = pixelX_start
        pixelY_curr += 1

In [12]:
# Hiding the encrypted data in the image
data = format(bytes_toInt(encrypted_data), "#0{0}b".format((len(encrypted_data) * 8) + 2))[2:]

for i in range(int(int(num_data_blocks, 2) * BLOCK_SIZE / 64)):
    # Finding a block to use
    found_block = False
    while not found_block:
        num_iterations += 1
        permutation_value = int(get_shaHash(permutation_seed, 512), 16)
        block_index = permutation_value % total_blocks
        blockX = block_index % num_blocksX
        blockY = int(block_index / num_blocksX)
        if blocks_used[blockY][blockX] == 0:
            found_block = True
            blocks_used[blockY][blockX] = 1
        else:
            permutation_seed = int_toBytes(bytes_toInt(permutation_seed) + num_iterations)
    rgb_index = permutation_value % 3
    pixelX_start = blockX * 8
    pixelX_curr = pixelX_start
    pixelY_start = blockY * 8
    pixelY_curr = pixelY_start
    
    
    update_seed_data = str(num_iterations)
    y = list(np.random.permutation(64))
    for i in range(64):
        a = int(y[i] / 8)
        b = int(y[i] % 8)
        pixelX_curr = a + pixelX_start
        pixelY_curr = b + pixelY_start
        pixel = img[pixelY_curr][pixelX_curr]
        pixel[rgb_index] = pixel[rgb_index] & ~1 | int(data[0])
        img[pixelY_curr][pixelX_curr] = pixel
        data = data[1:]
        if pixelX_curr == pixelY_curr:
            update_seed_data = update_seed_data + str(pixel[0] % 100) + str(pixel[1] % 100) + str(pixel[2] % 100)
        rgb_index = (rgb_index + 1) % 3
        
        
    permutation_seed = int_toBytes(int(update_seed_data) ^ bytes_toInt(permutation_seed))

assert len(data) == 0, "Some data was not hidden in the image"
cv2.imwrite(MODIFIED_IMAGE_PATH, img)

True

In [13]:
y = list(np.random.permutation(64))
print(y)

[54, 48, 63, 0, 13, 36, 34, 28, 25, 31, 6, 9, 26, 20, 19, 7, 32, 59, 22, 61, 16, 11, 50, 52, 30, 56, 39, 53, 8, 45, 15, 1, 60, 33, 23, 46, 29, 62, 47, 2, 4, 5, 3, 49, 17, 38, 35, 55, 44, 12, 10, 51, 58, 24, 37, 14, 43, 42, 41, 57, 27, 18, 40, 21]
